In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import cross_val_score

/Users/hsuan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import xgboost as xgb

In [8]:
data = pd.read_csv("WeatherData.csv", index_col=0)
data

,mintemp_c,maxtemp_c,precip_mm,sunshine,gust_kph,daily_will_it_rain,tom_will_it_rain,wind_kph,humidity,pressure_mb,...,temp_c,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI,Target
0,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,1,0,40,175.80,87.75,28.39,6,2.307144,-1.805114
1,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,2,0,74,170.76,84.59,29.01,2,3.343987,-1.806799
2,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,3,0,24,176.49,81.89,26.29,4,2.459838,-1.802157
3,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,4,0,39,175.88,88.58,28.64,0,4.243744,-1.807501
4,17.9,35.2,0.0,12.3,48,0,0,6,20,1006.3,...,26.6,5,0,61,173.37,89.91,29.91,1,4.302597,-1.810043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47575,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,6,1,37,162.47,74.20,28.11,5,2.384639,-0.963873
47576,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,7,1,30,162.40,70.60,26.77,2,3.139030,-0.964535
47577,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,8,1,34,162.24,69.47,26.39,1,2.461945,-0.962891
47578,22.9,33.4,17.8,6.2,93,1,1,15,67,1008.0,...,30.3,9,1,22,163.19,66.04,24.80,0,4.014209,-0.963780


In [9]:
#separate the response variable and predict variable
X = data.drop("Target", axis=1)
y = data["Target"]

#split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor()

# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'n_estimators': [400, 500, 550, 600],
    'learning_rate': [0.1],
    'max_depth': [7, 9, 11]
}

#find the best model and parameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=gbm_param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
print(best_model)

Best Parameters: {'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 600}
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=600, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


In [23]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 600}

In [24]:
#evaluate the best model
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

Mean Squared Error on Test Set: 4.568887122119206e-06


In [25]:
import pickle as pk

with open('xgboost2.model', 'wb') as file:
    pk.dump(best_model, file)

best: 300, 0.1, 9, 5.282754487174775e-06

In [48]:
import requests

url = "https://weatherapi-com.p.rapidapi.com/forecast.json"

querystring = {"q":"London","dt":"2023-12-01"}

headers = {
    "X-RapidAPI-Key": "811d87de6dmsha294081dc7cff1ep1dd89cjsn6c3c6c2da9c9",
    "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.52, 'lon': -0.11, 'tz_id': 'Europe/London', 'localtime_epoch': 1700891925, 'localtime': '2023-11-25 5:58'}, 'current': {'last_updated_epoch': 1700891100, 'last_updated': '2023-11-25 05:45', 'temp_c': 0.0, 'temp_f': 32.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 340, 'wind_dir': 'NNW', 'pressure_mb': 1022.0, 'pressure_in': 30.18, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 0, 'feelslike_c': -4.2, 'feelslike_f': 24.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 12.8, 'gust_kph': 20.5}, 'forecast': {'forecastday': [{'date': '2023-12-01', 'date_epoch': 1701388800, 'day': {'maxtemp_c': 4.1, 'maxtemp_f': 39.3, 'mintemp_c': 1.6, 'mintemp_f': 34.9, 'avgtemp_c': 2.8, 'avgtemp_f': 37.1, 'maxwind_mph': 11.0, 'maxwind_kph': 17.6,

In [49]:
pd.DataFrame(response.json()['current'])

,last_updated_epoch,last_updated,temp_c,temp_f,is_day,condition,wind_mph,wind_kph,wind_degree,wind_dir,...,precip_in,humidity,cloud,feelslike_c,feelslike_f,vis_km,vis_miles,uv,gust_mph,gust_kph
code,1700891100,2023-11-25 05:45,0.0,32.0,0,1000,8.1,13.0,340,NNW,...,0.0,86,0,-4.2,24.5,10.0,6.0,1.0,12.8,20.5
icon,1700891100,2023-11-25 05:45,0.0,32.0,0,//cdn.weatherapi.com/weather/64x64/night/113.png,8.1,13.0,340,NNW,...,0.0,86,0,-4.2,24.5,10.0,6.0,1.0,12.8,20.5
text,1700891100,2023-11-25 05:45,0.0,32.0,0,Clear,8.1,13.0,340,NNW,...,0.0,86,0,-4.2,24.5,10.0,6.0,1.0,12.8,20.5


In [45]:
pd.DataFrame(response.json()['forecast']['forecastday'])

,date,date_epoch,day,astro,hour
0,2023-11-30,1701302400,"{'maxtemp_c': 3.5, 'maxtemp_f': 38.3, 'mintemp...","{'sunrise': '07:42 AM', 'sunset': '03:56 PM', ...","[{'time_epoch': 1701302400, 'time': '2023-11-3..."


In [47]:
pd.DataFrame(response.json()['forecast']['forecastday']).loc[0,'day']

{'maxtemp_c': 3.5,
 'maxtemp_f': 38.3,
 'mintemp_c': 2.4,
 'mintemp_f': 36.2,
 'avgtemp_c': 3.1,
 'avgtemp_f': 37.6,
 'maxwind_mph': 7.2,
 'maxwind_kph': 11.5,
 'totalprecip_mm': 0.86,
 'totalprecip_in': 0.03,
 'totalsnow_cm': 0.73,
 'avgvis_km': 7.4,
 'avgvis_miles': 4.0,
 'avghumidity': 85.0,
 'daily_will_it_rain': 1,
 'daily_chance_of_rain': 83,
 'daily_will_it_snow': 0,
 'daily_chance_of_snow': 0,
 'condition': {'text': 'Patchy rain possible',
  'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png',
  'code': 1063},
 'uv': 1.0}

## Realtime

In [50]:
import requests

url = "https://weatherapi-com.p.rapidapi.com/current.json"

querystring = {"q":"53.1,-0.13"}

headers = {
	"X-RapidAPI-Key": "811d87de6dmsha294081dc7cff1ep1dd89cjsn6c3c6c2da9c9",
	"X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'location': {'name': 'Boston', 'region': 'Lincolnshire', 'country': 'United Kingdom', 'lat': 53.1, 'lon': -0.13, 'tz_id': 'Europe/London', 'localtime_epoch': 1700894272, 'localtime': '2023-11-25 6:37'}, 'current': {'last_updated_epoch': 1700893800, 'last_updated': '2023-11-25 06:30', 'temp_c': 2.0, 'temp_f': 35.6, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 12.5, 'wind_kph': 20.2, 'wind_degree': 310, 'wind_dir': 'NW', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 75, 'cloud': 25, 'feelslike_c': -3.2, 'feelslike_f': 26.2, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 22.1, 'gust_kph': 35.5}}


## Forecast

In [ ]:
import requests

url = "https://weatherapi-com.p.rapidapi.com/forecast.json"

querystring = {"q":"London","days":"6"}

headers = {
	"X-RapidAPI-Key": "811d87de6dmsha294081dc7cff1ep1dd89cjsn6c3c6c2da9c9",
	"X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())